In [1]:
import sys
import time
import pickle
from pathlib import Path

import numpy as np
import tensorflow as tf

from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import Callback

In [2]:
from sklearn.preprocessing import StandardScaler

In [3]:
sys.path.append(str(Path().resolve().parent))

In [4]:
from torsion.model import get_model

In [5]:
y_train = np.load('y_train.npy')
y_test = np.load('y_test.npy')
X_train = np.load('X_train.npy')
X_test = np.load('X_test.npy')

In [6]:
# Cap relative energies 
REL_ENERGY_CAP = 30.0
tmp_idx = y_train > REL_ENERGY_CAP
y_train[tmp_idx] = REL_ENERGY_CAP + np.exp(REL_ENERGY_CAP - y_train[tmp_idx])

In [7]:
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)

print('X_train.shape ', X_train.shape)

X_train.shape  (9600, 293)


In [8]:
scaler_file = 'scaler.pkl'
model_file = 'model.h5'

In [9]:
# save feature transformation
with open(scaler_file, 'wb') as fptr:
    pickle.dump(scaler, fptr)

In [10]:
_, num_feat = X_train.shape

# early stopping criteria
earlystop = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=100, \
                          verbose=1, mode='auto')

# create DNN model
model = get_model(num_feat)

In [11]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 2930)              861420    
_________________________________________________________________
batch_normalization (BatchNo (None, 2930)              11720     
_________________________________________________________________
dropout (Dropout)            (None, 2930)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1465)              4293915   
_________________________________________________________________
batch_normalization_1 (Batch (None, 1465)              5860      
_________________________________________________________________
dropout_1 (Dropout)          (None, 1465)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 293)               4

In [12]:
checkpointer = ModelCheckpoint(filepath=model_file, verbose=1, save_best_only=True)
callbacks_list = [checkpointer, earlystop]

# train DNN model
model.fit(
    X_train,
    y_train,
    epochs=5000,
    batch_size=256,
    validation_split=0.1,
    callbacks=callbacks_list,
    verbose=2)

print('Training complete')
print(f'Standard scalar is saved in {scaler_file}')
print(f'Model is saved in {model_file}')

Epoch 1/5000

Epoch 00001: val_loss improved from inf to 1.88996, saving model to model.h5
34/34 - 4s - loss: 1.6116 - val_loss: 1.8900
Epoch 2/5000

Epoch 00002: val_loss improved from 1.88996 to 1.68420, saving model to model.h5
34/34 - 1s - loss: 1.2783 - val_loss: 1.6842
Epoch 3/5000

Epoch 00003: val_loss improved from 1.68420 to 1.62709, saving model to model.h5
34/34 - 1s - loss: 1.1683 - val_loss: 1.6271
Epoch 4/5000

Epoch 00004: val_loss improved from 1.62709 to 1.53948, saving model to model.h5
34/34 - 1s - loss: 0.9984 - val_loss: 1.5395
Epoch 5/5000

Epoch 00005: val_loss improved from 1.53948 to 1.46478, saving model to model.h5
34/34 - 1s - loss: 0.9141 - val_loss: 1.4648
Epoch 6/5000

Epoch 00006: val_loss improved from 1.46478 to 1.42968, saving model to model.h5
34/34 - 1s - loss: 0.8053 - val_loss: 1.4297
Epoch 7/5000

Epoch 00007: val_loss improved from 1.42968 to 1.29315, saving model to model.h5
34/34 - 1s - loss: 0.7853 - val_loss: 1.2932
Epoch 8/5000

Epoch 0000